### This is a basic example of using 2 python packages:  pyds9 and pyregion.  I use pyds9 to plot image data in the ds9 viewer, and pyregion to manipulate ds9 regions files.

This example loads data from a fits file into a numpy ndarray, does math on that array, and plots the array in ds9, and then does some filtering and math using regions files.  By Jane Rigby, 6/2016, as a self-tutorial.  Use it if it helps you.

In [43]:
import numpy as np
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.io.fits import getdata
import pyds9
import pyregion

In [44]:
image_file = download_file('http://data.astropy.org/tutorials/FITS-images/HorseHead.fits', cache=True )
f = fits.open(image_file)
dum = f[0].data  # screwing around to get hdu as pyregions wants it
im = dum.astype(np.float64)
#(im_int, hdr) = getdata(image_file, header=True) #image is numpy array
im +=0.01 # im is a numpy array, so we can do math on it.
#d = pyds9.DS9('foo1')  # start ds9.  'd' is the way to call ds9
d = pyds9.DS9('/var/folders/ly/n6803yys5cs9tw41zytqgmvcmz2qhx/T//DS9_foo1.39834') #kludge
d.set_np2arr(im) # sending ndarray im directly to ds9
d.set("colorbar no")   # example of manipulating the ds9 window
d.set("scale zscale")  # example of manipulating the ds9 window
d.set("zoom to 0.6 0.6")

1

In [45]:
## As an example of using pyregion, do some stupid aperture photometry
reg = 'image;circle(727.0,603.0,21)' # example region
bkg = 'image;circle(800.0,600.0,21)' # bkg region
d.set('regions', reg)  # load that region in ds9
d.set('regions', bkg)
r = pyregion.parse(reg)               # Ugly but works, 10pm.  get_mask syntax is failing
myfilter = r.get_filter()
total_flux = np.nansum(myfilter.mask(im) * im) 
#
b = pyregion.parse(bkg)
bkgfilter = b.get_filter()
bkg_flux = np.nansum(bkgfilter.mask(im) * im)
print "Stupid aperture photometry:"
print "flux_in_aperture  flux_in_bkg   net_flux"
print total_flux, bkg_flux, total_flux - bkg_flux

Stupid aperture photometry:
flux_in_aperture  flux_in_bkg   net_flux
18665478.73 16994411.73 1671067.0
